## **2. Tokenizacja zbiorów danych**

### **Uwaga!**
Ze względu na poufność danych, surowe raporty i adnotacje ekspertów nie są zawarte w tym repozytorium.

### **Problem:** 
Wieloetykietowa klasyfikacja długich, nieustrukturyzowanych dokumentów w języku polskim, w warunkach silnego niezbalansowania klas.

### **Model enkodera:** sdadas/polish-longformer-base-4096 (https://huggingface.co/sdadas/polish-longformer-base-4096)

#### **Import Bibliotek**

In [1]:
import os
import pandas as pd
from transformers import LongformerTokenizerFast
from datasets import Dataset, DatasetDict

Ustawienie sztywnych parametrów dla okna kontekstowego zgodnie z dokumentacją modelu Bielik-1.5B-v3 oraz narzucenie wymiaru okna przesuwnego.

In [2]:
MODEL_NAME = "sdadas/polish-longformer-base-4096"
MAX_LENGTH = 4096
STRIDE = 512

In [3]:
df_train = pd.read_json('data/data_split/train.jsonl', lines=True)
df_val = pd.read_json('data/data_split/validation.jsonl', lines=True)
df_test = pd.read_json('data/data_split/test.jsonl', lines=True)

#### **Przygotowanie do tokenizacji.**

In [4]:
train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)

raw_datasets = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

Wczytanie odpowiedniego tokenizera

In [5]:
tokenizer = LongformerTokenizerFast.from_pretrained(MODEL_NAME)

#### **Przeprowadzenie tokenizacji.**

In [6]:
def chunk_and_tokenize(examples):
    tokenized_output = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH,
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=False,
    )

    # Pobieramy mapowanie z nowych, podzielonych fragmentów do oryginalnych przykładów
    sample_mapping = tokenized_output.pop("overflow_to_sample_mapping")
    
    # Tworzymy listę etykiet dla każdego nowego fragmentu i dodajemy numer id
    labels = []
    doc_ids = []
    for i in range(len(tokenized_output["input_ids"])):
        original_sample_idx = sample_mapping[i]
        labels.append(examples["labels"][original_sample_idx])
        doc_ids.append(examples["doc_id"][original_sample_idx])
    
    tokenized_output["labels"] = labels
    tokenized_output["doc_id"] = doc_ids
    return tokenized_output

In [7]:
tokenized_datasets = raw_datasets.map(
    chunk_and_tokenize,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
    num_proc=8
)

Map (num_proc=8):   0%|          | 0/275 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/59 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/59 [00:00<?, ? examples/s]

#### **Zapis wyników tokenizacji.**

In [8]:

OUTPUT_DIR = "data/data_tokenized"
os.makedirs(OUTPUT_DIR, exist_ok=True)

tokenized_datasets.save_to_disk(OUTPUT_DIR)

Saving the dataset (0/1 shards):   0%|          | 0/5070 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1062 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/929 [00:00<?, ? examples/s]